In [1]:
import time
import random

import pandas as pd
from tqdm.auto import tqdm
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

In [2]:
url = "https://www.wadiz.kr/web/wreward/main"

# Chrome WebDriver 경로 설정
driver = webdriver.Chrome()

# 웹사이트 열기
driver.get(url)

# 페이지 로드를 기다림
driver.implicitly_wait(10)

tabs_mobile_dict = {
    '전체': 1, 'best펀딩': 2, '테크, 가전': 3, 
    '패션': 4, '뷰티': 5, '홈, 리빙': 6,
    '스포츠, 아웃도어': 7, '푸드': 8, '도서, 전자책': 9, 
    '클래스': 10, '디자인': 11, '반려동물': 12, 
    '아트': 13, '캐릭터, 굿즈': 14, '영화, 음악': 15, 
    '키즈': 16, '게임': 17, '만화, 웹툰': 18, 
    '핸드메이드': 19, '사전': 20, '여행': 21, 
    '자동차': 22, '멤버십': 23, '소셜': 24
    }

# XPath를 이용해 요소 찾기
tabs_mobile = tabs_mobile_dict['테크, 가전']
tabs_mobile_element = driver.find_element(By.XPATH, f'//*[@id="main-app"]/div[2]/div[4]/div/div/div/ul/li[{tabs_mobile}]/button')
# 요소 클릭
tabs_mobile_element.click()
# time.sleep(1)

# 페이지 로드를 기다림
driver.implicitly_wait(10)

In [3]:
# 드롭다운 요소 식별
dropdown = driver.find_element(By.CLASS_NAME, "OrderSelect_selectBox__2hBjk") # 전체, 진행중, 종료된

# Select 객체 생성
select = Select(dropdown)

# 옵션 선택 방법
dropdown_select_dict = {'0전체': "전체", "1진행중": "진행중", "2종료된": "종료된"}

# 원하는 옵션 선택
select.select_by_visible_text(dropdown_select_dict["1진행중"])

# 페이지 로드를 기다림
driver.implicitly_wait(10)

In [4]:
# 정렬 순서 변경
sort_criteria_dict = {'추천순': 1, '인기순': 2, '모집금액순': 3, '마감임박순': 4, '최신순': 5, '응원참여순': 6}
sort_num = sort_criteria_dict['인기순']
sort_criteria_element = driver.find_element(By.XPATH, f'//*[@id="main-app"]/div[2]/div[6]/div[1]/div/ul/li[{sort_num}]')

# 요소 클릭
sort_criteria_element.click()
# time.sleep(1)

# 페이지 로드를 기다림
driver.implicitly_wait(10)


In [5]:
# 더이상 로딩이 되는 것 없을 때까지 스크롤 다운 반복
last_height = driver.execute_script("return document.body.scrollHeight")

scroll_count = 0 #스크롤 몇번 했는지 세어주는 것

while True:
    # 현재 페이지 기준 끝까지 아래로 스크롤 다운하기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 페이지 로드를 기다림
    driver.implicitly_wait(10)
    time.sleep(1)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print(f"**break** {scroll_count}번 스크롤링, 최대{48*scroll_count}개 로딩됨", flush=True)
        break
    
    last_height = new_height
    scroll_count += 1
    
    if scroll_count >21:
        print(f"너무 데이터가 많아서 {48*scroll_count}개까지만 했습니다", flush=True)

**break** 3번 스크롤링, 최대144개 로딩됨


In [6]:
# 현재 페이지 제일 위로 스크롤 업하기
driver.execute_script("window.scrollTo(document.body.scrollHeight, 0);")

In [7]:
# 데이터 긁어오기

data = []

# 기본적으로 로딩 되는 것이 1~48까지 48개 로딩됨, 그리고 추가 로딩 되면 48개씩 추가 로딩됨
expected_end_num = 48*(scroll_count+1) 

# 찾기 작업을 수행하는 함수
def safe_find(soup, number, tag, class_=None):
    try:
        result = soup.find(tag, class_=class_)
        return result.text if result else None
    except Exception as e:
        print(f"{number}번 째 {tag}// {class_}에서 예외 발생: {e}")
        return None

# select 메소드를 사용하는 작업을 수행하는 함수
def safe_select(soup, number, selector, index=0):
    try:
        results = soup.select(selector)
        if results and len(results) > index:
            return results[index].text
        else:
            return None
    except Exception as e:
        print(f"{number}번 째 {selector}에서 예외 발생: {e}")
        return None

for number in tqdm(range(1, expected_end_num)): #11
    scraped_data = {
        'big_tag': None,
        'hash_tag': None,
        'funding_type': None,
        'funding_type_secondary': None,
        'title': None,
        'description': None,
        'num_of_participants': None,
        'left_day': None,
        'amount': None,
        'achieve_rate': None,
        'url': None,
    }
    try:
        # XPath를 이용해 요소 찾기
        content_element = driver.find_element(By.XPATH, f'//*[@id="main-app"]/div[2]/div[6]/div[2]/div[{number}]/a')
        
        # 컨텐츠 요소 클릭
        content_element.click()
        time.sleep(random.uniform(1,3))
        # 페이지 로드를 기다림
        driver.implicitly_wait(10)
        
        # 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # print(soup.prettify())
        
        # 프리오더이냐 펀딩이냐에 따라서 태그의 값들이 다른 경우가 있음
        
        # 어느 것인지 확인하는 구분자
        parsing_sep = soup.select_one(".CommonInfo_container__1J2gi").find("div").get("class")[0]
        
        
        # 프리오더/ 펀딩에 따라 태그 다른 값들
        if parsing_sep == "BasePreOrderInfo_container__2TMxc": # 프리오더
            scraped_data['funding_type'] = safe_find(soup, number=number, tag="span", class_="FundingTypeBadge_preorder__20AX-")
            scraped_data['funding_type_secondary'] = safe_find(soup, number, tag="span", class_="Badge_label__3rWfq")
            scraped_data['title'] = safe_find(soup, number, tag="p", class_="BasePreOrderInfo_title__jxsg5")
            scraped_data['description'] = safe_find(soup, number, tag="p", class_="BasePreOrderInfo_description__LxWkO")
            scraped_data['num_of_participants'] = safe_select(soup, number, selector='.BasePreOrderInfo_infoWrapper__3wo9k span', index=0)
            scraped_data['left_day'] = safe_select(soup, number, selector='.BasePreOrderInfo_rate__2uwso span span', index=0)
            scraped_data['amount'] = safe_find(soup, number, tag="p", class_="BasePreOrderInfo_totalBackedAmount__2FNmg")
            scraped_data['achieve_rate'] = safe_select(soup, number, selector='.BasePreOrderInfo_rate__2uwso span span', index=1)
        
        elif parsing_sep == "BaseFundingInfo_container__sotrP": #펀딩
            scraped_data['funding_type'] = safe_find(soup, number, tag="span", class_="FundingTypeBadge_funding__4BhpK")
            scraped_data['funding_type_secondary'] = "없음" # None
            scraped_data['title'] = safe_find(soup, number, tag='p', class_="BaseFundingInfo_title__1Dw42")
            scraped_data['description'] = safe_find(soup, number, tag="p", class_="BaseFundingInfo_description__1XXyy")
            scraped_data['achieve_rate'] = safe_select(soup, number, selector='.BaseFundingInfo_infoWrapper__23LoT span', index=0)
            scraped_data['left_day'] = safe_select(soup, number, selector='.BaseFundingInfo_rate__1uMoD span span', index=0)
            scraped_data['amount'] = safe_find(soup, number, tag="p", class_="BaseFundingInfo_totalBackedAmount__1ezH_")
            scraped_data['num_of_participants'] = safe_select(soup, number, selector='.BaseFundingInfo_rate__1uMoD span span', index=1)
            
        
        else:
            raise ValueError(f"확인한 것과 다른 구분자가 확인되었습니다 : {parsing_sep}")
        # 공통적인 것, 예외처리 안해줬음
        scraped_data['big_tag'] = soup.find('span', {'aria-label': '카테고리'}).text
        scraped_data['hash_tag'] = soup.select('.ProjectSearchTags_hashTag__2UMo_ span')[1].text
        scraped_data['url'] = driver.current_url
        
        data.append(scraped_data)
        
        # 되돌아가기
        driver.back()
        time.sleep(random.uniform(1,3))
    
    except NoSuchElementException:
        print(f"요소를 찾을 수 없습니다. {number}를 찾지 못했습니다") #, flush=True
        break  # 요소를 찾지 못하면 for문 중단
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")
        print(f"{number}째에서 발생했습니다: {e}")
        continue  # 다음 항목으로 계속 진행

  0%|          | 0/191 [00:00<?, ?it/s]

요소를 찾을 수 없습니다. 109를 찾지 못했습니다


In [8]:
# WebDriver 종료
driver.quit()

In [9]:
# 데이터 프레임 생성
df_sights = pd.DataFrame(data)

# 데이터 프레임 출력
df_sights

,big_tag,hash_tag,funding_type,funding_type_secondary,title,description,num_of_participants,left_day,amount,achieve_rate,url
0,생활가전,청소,펀딩,없음,[먼지킬러] 분당 9000번! 강력진동으로 먼지를 제거합니다! 제트브러시,봄맞이 대청소! 청소혁명! 집안 구석구석 쌓인 먼지를 한번에 싹! 세계최초 3차원 ...,556명 참여,19일 남음,"36,915,010원 달성","3,691",https://www.wadiz.kr/web/campaign/detail/16209...
1,주변기기,테크,프리오더-글로벌,국내단독,"[글로벌10억] 접으면 무선 마우스, 펼치면 어디든 함께! 외근 필수템","[일본미국 10억 펀딩] 장담해요. 이제 더 이상 플라스틱 마우스를 떨어뜨릴 일도,...","4,050",29일 남음,"338,956,501원 달성","33,895% 달성",https://www.wadiz.kr/web/campaign/detail/27695...
2,엔터테인먼트가전,선물,펀딩,없음,"[CES2024] 착-붙 메모지 프린터, GPT를 만나 더욱 똑똑해졌어요","세상에서 가장 똑똑한 미니 프린터 네모닉AI!\n누적 200억, CES 최고혁신상,...","1,356명 참여",27일 남음,"105,789,500원 달성","10,578",https://www.wadiz.kr/web/campaign/detail/27556...
3,생활가전,선풍기,프리오더,특별구성,1~100단까지 진짜 강력한 바람 등장! 파워 휴대용 에어컨 선풍기,"와디즈 1위 휴대용 선풍기, 에어컨 선풍기가 바람이 더 강력해져 돌아왔습니다.\n1...",875,16일 남음,"41,325,400원 달성","8,265% 달성",https://www.wadiz.kr/web/campaign/detail/27665...
4,웨어러블,충전기,프리오더-앵콜,특별구성,2만원대 넣을수 없는 기능을 모두 담았습니다｜맥세이프 보조배터리혁명,무겁고 투박한 보조배터리는 이제 그만! 어디에서도 볼 수 없는 유니크한 디자인과 2...,862,2일 남음,"30,740,200원 달성","6,148% 달성",https://www.wadiz.kr/web/campaign/detail/27668...
...,...,...,...,...,...,...,...,...,...,...,...
103,테크·가전,멀티,펀딩,없음,다이어트 실패할 확률 적어요! 성공비결은 더블 임팩트 체중계~,[집에 모셔둔 체중계는 모두 버리자! ] \n당신의 다이어트 작심삼일을 박살내는 ...,2명 참여,49일 남음,"27,900원 달성",0,https://www.wadiz.kr/web/campaign/detail/27834...
104,주방가전,주방용품,프리오더-앵콜,특별구성,온풍살균 자외선살균 으로 주방 식기구 모두 뽀송뽀송하게 살균 하세요,"스마트살균기 로 주방에 품격을 높이고 수저,젓가락 칼 행주등 살균하세요 국내에서...",1,3일 남음,0원 달성,0% 달성,https://www.wadiz.kr/web/campaign/detail/27763...
105,주방가전,드립커피,프리오더-앵콜,국내단독,업그레이드된 모터로 성능이 업그레이드되어 돌아온 홈카페 그라인더 끝판왕!,"모터 정밀기어박스 업그레이드로 성능은 더 강력해졌지만, 가격은 더 착해진 58esp...",0,8일 남음,0원 달성,0% 달성,https://www.wadiz.kr/web/campaign/detail/27912...
106,생활가전,캠핑,프리오더-글로벌,국내단독,발받침대 기능 총집합. 바른자세와 편안함을 동시에 스마트 발 받침대!,기존의 발 받침대들의 기능들을 통폐합했습니다!높이 조절 가능한 발바닥 및 푹신한 다...,0,22일 남음,0원 달성,0% 달성,https://www.wadiz.kr/web/campaign/detail/28238...


In [10]:
df_sights.to_parquet("./크롤링/20240420테크가전.parquet")